In [0]:
from pyspark.sql import functions as F, Window
from helpers import BRONZE, SILVER, TAXI_TYPES
spark.conf.set(
    "spark.databricks.delta.properties.defaults.feature.timestampNtz",
    "supported"
)

In [0]:
def enrich_base(df, p_col, d_col):
    return (df
            .withColumn(
                "trip_duration_min",
                F.expr(f"timestampdiff(SECOND, {p_col}, {d_col})")/60.0
            )
            .withColumn("pickup_hour",  F.hour(p_col))
            .withColumn("weekday",      F.date_format(p_col, "E"))
            .withColumn("week_of_year", F.weekofyear(p_col))
           )

In [0]:
for tt in TAXI_TYPES:
    src_tbl = f"{BRONZE.SCHEMA}.{tt}_tripdata_bronze"
    if not spark.catalog.tableExists(src_tbl):
        print(f"Camada bronze sem dados para {tt}, pulando…")
        continue

    df = spark.table(src_tbl)

    pickup_col, drop_col = (
        ("tpep_pickup_datetime", "tpep_dropoff_datetime")
        if tt in ("yellow", "green")
        else ("pickup_datetime",  "dropoff_datetime")
    )

    df = enrich_base(df, pickup_col, drop_col)

    rules = (
        (F.col("total_amount")    >= 0) &
        (F.col("trip_distance")   >  0) & (F.col("trip_distance") < 100) &
        (F.col(pickup_col)        <  F.col(drop_col))
    )
    if "passenger_count" in df.columns:
        rules &= (
            F.col("passenger_count").isNull() |
            ((F.col("passenger_count") >= 1) & (F.col("passenger_count") <= 8))
        )

    period_valid = (
        (F.col("trip_year") == 2023) &
        (F.col("trip_month").between(1, 5))
    )

    df = df.filter(rules & period_valid)

    tgt_tbl = f"{SILVER.SCHEMA}.{tt}_tripdata_silver"

    (df.write
        .format("delta")
        .mode("overwrite")                
        .partitionBy("trip_year", "trip_month")
        .option("overwriteSchema", "true")
        .option("delta.feature.checkConstraints", "supported")
        .option("delta.constraints.amount_nonneg",      "total_amount >= 0")
        .option("delta.constraints.distance_positive",  "trip_distance > 0 AND trip_distance < 100")
        .option("delta.constraints.pickup_before_drop", f"{pickup_col} < {drop_col}")
        .option("delta.constraints.passenger_ok",
                "passenger_count IS NULL OR (passenger_count BETWEEN 1 AND 8)")
        .option("delta.constraints.period_2023_jan_may",
                "trip_year = 2023 AND trip_month BETWEEN 1 AND 5")
        .saveAsTable(tgt_tbl) 
    )

    print(f"Silver {tt}: {spark.table(tgt_tbl).count():,} linhas")

print("Camada Silver processada!")
